# Data augmentation notebook  
This notebook will perform data augmentation on the provided training set.  
The goal of data augmentation is to have additional images in order to
- correctly train our classifier
- have more diverse images  

Data augmentation will be performed this way :  
Each training image and its corresponding ground truth will be :
1. kept as original (ori)
2. horizontally flipped (hflip)
3. vertically flipped (vflip)
4. 45° rotated (rot45)
5. 90° rotated (rot90)
6. 135° rotated (rot135)
7. 180° rotated (rot180)
8. blurred using a gaussian filter (blur)
9. lightened (light)
10. darkened (dark)

The final augmented training set will therefore be composed of 1000 satellite images and correspondingly 1000 ground truths.

## Imports

In [31]:
import scipy.ndimage as ndi
import cv2
import numpy as np
from helpers import load_training_data, img_float_to_uint8

## Loading provided training images

In [34]:
# Loading training images and corresponding ground truth
data_dir = '../data/training/'
imgs, gndt = load_training_data(data_dir)

Loading 100 images


PermissionError: [Errno 13] Permission denied: '../data/training/images/.ipynb_checkpoints'

In [3]:
print("Value range of images: [{}, {}] and ground truths: [{}, {}]".format(np.min(imgs), np.max(imgs), np.min(gndt), np.max(gndt)))

Value range of images: [0.0, 1.0] and ground truths: [0.0, 0.929411768913269]


In [4]:
# Scale images dimensions to [0;255] and convert to 8-bit
n = len(imgs)
imgs8 = [img_float_to_uint8(imgs[i]) for i in range(n)]
gndt8 = [img_float_to_uint8(gndt[i]) for i in range(n)]

In [5]:
print("Value range of scaled images: [{}, {}] and ground truths: [{}, {}]".format(np.min(imgs8), np.max(imgs8), np.min(gndt8), np.max(gndt8)))

Value range of scaled images: [0, 255] and ground truths: [0, 255]


In [6]:
save_dir = "../data/training_augmented/"
dir_imgs = save_dir + "images/"
dir_gt = save_dir + "groundtruth/"

### 1. Save original images

In [13]:
for i in range(n):
    cv2.imwrite(dir_imgs + 'ori_' + str(i+1).zfill(3) + '.png', imgs8[i])
    cv2.imwrite(dir_gt + 'ori_' + str(i+1).zfill(3) + '.png', gndt8[i])

### 2. Horizontally flipped images

In [24]:
for i in range(n):
    hflip_img = np.flip(imgs8[i], 1)
    hflip_gt = np.flip(gndt8[i], 1)
    cv2.imwrite(dir_imgs + 'hflip_' + str(i+1).zfill(3) + '.png', hflip_img)
    cv2.imwrite(dir_gt + 'hflip_' + str(i+1).zfill(3) + '.png', hflip_gt)

### 3. Vertically flipped images

In [25]:
for i in range(n):
    vflip_img = np.flip(imgs8[i], 0)
    vflip_gt = np.flip(gndt8[i], 0)
    cv2.imwrite(dir_imgs + 'vflip_' + str(i+1).zfill(3) + '.png', vflip_img)
    cv2.imwrite(dir_gt + 'vflip_' + str(i+1).zfill(3) + '.png', vflip_gt)

### 4, 5, 6, 7. Rotated images

In [19]:
for ang in [45.0, 90.0, 135.0, 180.0]:
    for i in range(n):
        rot_im = ndi.rotate(imgs8[i], ang, reshape=False, mode='reflect')
        rot_gt = ndi.rotate(gndt8[i], ang, reshape=False, mode='reflect')
        cv2.imwrite(dir_imgs + 'rot' + str(np.int64(ang)) + '_' + str(i+1).zfill(3) + '.png', rot_im)
        cv2.imwrite(dir_gt + 'rot' + str(np.int64(ang)) + '_' + str(i+1).zfill(3) + '.png', rot_gt)

### 8. Gaussian blur

note : blurred images should not change the detected road, so the ground truths won't be blurred.

In [38]:
for i in range(n):
    blur_img = cv2.GaussianBlur(imgs8[i], ksize = (3,3), sigmaX = 0, borderType = cv2.BORDER_REFLECT)
    cv2.imwrite(dir_imgs + 'blur_' + str(i+1).zfill(3) + '.png', blur_img)
    cv2.imwrite(dir_gt + 'blur_' + str(i+1).zfill(3) + '.png', gndt8[i])

### 9, 10. Brightness change

note : lighter or darker images should not change the detected road, so the ground truths won't be modified.

In [21]:
for i in range(n):
    light = cv2.convertScaleAbs(imgs8[i], beta=40) 
    dark = cv2.convertScaleAbs(imgs8[i], beta=-40) 
    cv2.imwrite(dir_imgs + 'light_' + str(i+1).zfill(3) + '.png', light)
    cv2.imwrite(dir_gt + 'light_' + str(i+1).zfill(3) + '.png', gndt8[i])
    cv2.imwrite(dir_imgs + 'dark_' + str(i+1).zfill(3) + '.png', dark)
    cv2.imwrite(dir_gt + 'dark_' + str(i+1).zfill(3) + '.png', gndt8[i])